# Προεπεξεργασία

#### Το τετράδιο αυτό φτιάχτηκε για το πρώτο μέρος της εργασίας, που είναι η προεπεξεργασία των δεδομένων με τα οποία θα εκπαιδεύσουμε αργότερα το νευρωνικό δίκτυο.

#### Αρχικά, ορίζουμε τις βιβλιοθήκες που θα χρειαστούμε για την προεπεξεργασία. Η βιβλιοθήκη __time__ βοηθάει στη χρονομέτρηση εκτέλεσης κάποιου μέρους του κώδικα. Οι βιβλιοθήκες __numpy__ και __pandas__ κάνουν πιο εύκολη την προεπεξεργασία. Τέλος, η __sklearn__ χρησιμοποιήθηκε για την κανονικοποίηση.

In [1]:
import time
import numpy
import pandas
import sklearn.preprocessing

#### Είμαστε πλέον έτοιμοι για την εισαγωγή των δεδομένων στο πρόγραμμά μας. Τα δεδομένα μας θα τα εισάγουμε με τη μορφή dataframe, έτσι ώστε να μπορούμε να τα διαχειριστούμε πιο εύκολα. To αρχείο μας ονομάζεται __"u.data"__ και παρατηρούμε ότι το τροφοδοτούμε με __relative path__ στη συνάρτηση __read_csv__. Αν το αρχείο δεν είναι στον ίδιο φάκελο με το παρόν τετράδιο, τότε θα πρέπει να αντικατασταθεί η __relative path__ μορφή που υπάρχει παρακάτω από το __absolute path__ του αρχείου __"u.data"__. Επίσης, χρησιμοποιούμε ως διαχωριστικό χαρακτήρα το __tab__, λόγω της δομής του αρχείου. Τέλος, δεν ορίζουμε κεφαλίδες ακόμα για τις στήλες του __dataframe__.

In [2]:
df_data = pandas.read_csv( "u.data", delimiter = "\t", header = None )

#### Έχοντας εισάγει στο πρόγραμμά μας το σύνολο δεδομένων που θέλουμε να επεξεργαστούμε, ορίζουμε τα ονόματα των στηλών του __dataframe__. Έτσι, το __indexing__ θα γίνει ευκολότερο μετά.

In [3]:
df_data.columns = ["userid", "movieid", "rating", "timestamp"]

#### Πατατηρούμε από την άσκηση ότι το __column timestamp__ δε θα μας χρειαστεί, κι άρα μπορούμε να το διαγράψουμε.

In [4]:
df_data.drop("timestamp", inplace = True, axis = 1)

#### Διατάσουμε τις σειρές σύμφωνα με αύξουσα σειρά και κριτήριο τη στήλη __userid__. Αυτό θα μας επιτρέψει στη συνέχεια να κάνουμε __προχωρημένο indexing__.

In [5]:
df_data_sorted_by_user = df_data.sort_values(by=['userid'])

#### Το datraframe μας είναι όπως παρακάτω.

In [6]:
df_data_sorted_by_user

,userid,movieid,rating
66567,1,55,5
62820,1,203,4
10207,1,183,5
9971,1,150,5
22496,1,68,4
...,...,...,...
96823,943,427,4
70902,943,12,5
84518,943,284,2
72321,943,62,3


#### Τώρα θα ομαδοποιήσουμε τις σειρές που έχουν κοινό __userid__ κάνοντας έτσι ακόμα ευκολότερη τη διαχείρηση του dataframe. Αυτό είναι το βήμα του προχωρημένου indexing.

In [7]:
df_data_indexed_by_user = df_data_sorted_by_user.set_index(['userid', 'movieid'])

#### Οπότε πλέον το dataframe μας είναι όπως παρακάτω.

In [8]:
df_data_indexed_by_user

rating
userid movieid        
1      55            5
       203           4
       183           5
       150           5
       68            4
...                ...
943    427           4
       12            5
       284           2
       62            3
       210           4

[100000 rows x 1 columns]

#### Ένα από τα ερωτήματα της άσκησης είναι και η __αφαίρεση του μέσου__ όρου από όλες τις (δοσμένες) βαθμολογίες. Για να γίνει αυτό, θα υπολογήσουμε αρχικά από το dataframe που έχουμε το μέσο rating για κάθε χρήστη. 

In [9]:
df_mean_ratings = df_data_indexed_by_user.mean(level='userid')

#### Οι μέσοι όροι ανά χρήστη είναι όπως παρακάτω.

In [10]:
df_mean_ratings

,rating
userid,
1,3.610294
2,3.709677
3,2.796296
4,4.333333
5,2.874286
...,...
939,4.265306
940,3.457944
941,4.045455


#### Τώρα περνάμε στο ερώτημα της __συμπλήρωσης__ των __ελλιπών τιμών__ με __0__. Ο τρόπος που επιλέχθηκε να γίνει είναι με πίνακα της βιβλιοθήκης __numpy__. Παρακάτω αρχικοποιούμε τις διαστάσεις του πίνακα.

In [11]:
np_indexes = numpy.array(numpy.meshgrid(numpy.arange(1, df_mean_ratings.size + 1), numpy.arange(1, df_data_indexed_by_user.groupby('movieid').size().index.size + 1))).T.reshape(-1,2)

#### Έχοντας αρχικοποιήσει τον πίνακα που θέλαμε, τον μετατρέπουμε σε τύπο __typle__ για να το μετατρέψουμε μετά σε dataframe.

In [12]:
tuples = tuple(map(tuple, np_indexes))

#### Μετατρέπουμε το __tuple__ που φτιάξαμε πριν σε __dataframe__.

In [13]:
indexes = pandas.MultiIndex.from_tuples(tuples, names=['userid', 'movieid'])

#### Μετατρέπουμε το __dataframe__ σε τύπο δεδομένων __series__. Πειραματικά κατέληξα στο συμπέρασμα ότι η διαχείρηση των δεδομένων με __Pandas Series__ είναι ελάχιστα πιο γρήγορη από τη διαχείρηση των δεδομένων με __Pandas Dataframes__.

In [14]:
s_data = pandas.Series(0, index=indexes)

#### Εκτυπώνουμε τη μεταβλητή μας για να έχουμε εικόνα του τι έχουμε φτιαάξει.

In [15]:
s_data

userid  movieid
1       1          0
        2          0
        3          0
        4          0
        5          0
                  ..
943     1678       0
        1679       0
        1680       0
        1681       0
        1682       0
Length: 1586126, dtype: int64

#### Είμαστε έτοιμοι να ξανασυνεχίσουμε με dataframe. Κάνουμε οπότε __μετατροπή__ της μεταβλητής τύπου __series σε dataframe__.

In [16]:
df_zeros = s_data.to_frame().rename(columns={0: "rating"})

#### Κάνουμε συγχώνευση του dataframe που έχει τις κριτικές με αυτό που έχει συμπληρωμένες τις ζητούμενες τιμές . Υπενθυμίζεται πως ο τρόπος συμπλήρωσης που επιλέχθηκε είναι συμπλήρωση με 0.

In [17]:
df_data_merged = pandas.merge(df_zeros, df_data_indexed_by_user, how='left', on=['userid', 'movieid'])

#### Μετά από τη συγχώνευση, σχηματίζουμε και πάλι όπως θέλουμε το dataframe μας.

In [18]:
df_data_merged.drop("rating_x", inplace = True, axis = 1)
df_complete = df_data_merged.fillna(0)
df_complete = df_complete.rename(columns={"rating_y": "rating"})

#### Για να γίνει πιο εύκολα το κεντράρισμα, αρχικοποιείται ένας πίνακας με όλα τους μέσους όρους των χρηστών. Ο πίνακας έχει διαστάσεις __(αριθμός όλων των διακριτών χρηστών) x (αριθμό όλων των διακριτών ταινιών)__.

In [19]:
np_zeros = numpy.empty([df_mean_ratings.size + 1, df_data_indexed_by_user.groupby('movieid').size().index.size], dtype=float)
for i in numpy.arange(1, df_mean_ratings.size + 1):
    np_zeros[i] = numpy.repeat(float(df_mean_ratings.loc[i]), 1682)
np_mean_ratings = numpy.delete(np_zeros, 0, 0)

#### Ο πίνακας που φτιάξαμε φαίνεται παρακάτω.

In [20]:
np_mean_ratings

array([[3.61029412, 3.61029412, 3.61029412, ..., 3.61029412, 3.61029412,
        3.61029412],
       [3.70967742, 3.70967742, 3.70967742, ..., 3.70967742, 3.70967742,
        3.70967742],
       [2.7962963 , 2.7962963 , 2.7962963 , ..., 2.7962963 , 2.7962963 ,
        2.7962963 ],
       ...,
       [4.04545455, 4.04545455, 4.04545455, ..., 4.04545455, 4.04545455,
        4.04545455],
       [4.26582278, 4.26582278, 4.26582278, ..., 4.26582278, 4.26582278,
        4.26582278],
       [3.41071429, 3.41071429, 3.41071429, ..., 3.41071429, 3.41071429,
        3.41071429]])

#### Το επόμενο βήμα είναι να εντοπίσουμε τις κριτικές που έδωσαν οι χρήστες, και να αφαιρέσουμε από αυτές τη μέση βαθμολογία που αντιστοιχεί στο συγκεκριμένο χρήστη. Ουσιαστικά, παρακάτω γίνεται το __κεντράρισμα__. Κατά μέσο όρο, επειδή έχουμε να επεξεργαστούμε μεγάλο όγκο δεδομένων, και η python ειναι αργή γλώσσα, υπολογίζεται ότι __η παρακάτω διαδικασία θα διαρκέσει περίπου 10 λεπτά__.

In [21]:
start = time.time()
for i,j in df_complete.index:
    if df_complete.loc[(i,j), 'rating'] != 0:
        df_complete.loc[i,j] = df_complete.loc[(i,j), 'rating'] - np_mean_ratings[i - 1][j - 1]
end = time.time()
print("Total time: ", int(end - start) + 1, " seconds.")

Total time:  298  seconds.


#### Μετά από τη συμπλήρωση των ελλειπών τιμών και το κεντράρισμα, είμαστε σε θέση να __κάνουμε__ την __κανονικοποίηση__. Η κανονικοποίηση γίνεται στη στήλη των __ratings__.

In [22]:
ratings = df_complete.values
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
ratings_scaled = min_max_scaler.fit_transform(ratings)
df_complete.loc[:, ('rating')] = ratings_scaled

#### Σε αυτό το σημείο έχει τελειώσει η προεπεξεργασία των δεδομένων. Το splitting για __το 5-fold cross validation γίνεται σε επόμενο τετράδιο__.

In [23]:
df_complete.to_csv('/home/andreas/data/complete_normalized_data.csv', encoding='utf-8')

#### Παρακάτω μετατρέπουμε σε μορφή __μητρώου__ τα δεδομένα μας.

In [24]:
df_tabular = df_complete.reset_index().pivot(index='userid', columns='movieid', values='rating')

#### Το μητρώο που δημιουργήσαμε φαίνεται παρακάτω.

In [25]:
df_tabular

movieid,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userid,,,,,,,,,,,,,,,,,,,,,
1,0.707095,0.430554,0.568825,0.430554,0.430554,0.707095,0.568825,0.154012,0.707095,0.430554,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
2,0.555083,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.278541,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
3,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
4,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
5,0.670593,0.532322,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.514940,0.616526,0.514940,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
940,0.514940,0.514940,0.514940,0.313349,0.514940,0.514940,0.589890,0.728161,0.451619,0.514940,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
941,0.646925,0.514940,0.514940,0.514940,0.514940,0.514940,0.508655,0.514940,0.514940,0.514940,...,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494,0.51494
